# imports

In [1]:
library(tidyverse)
library(dplyr)
#install.packages("Metrics")
library(Metrics)
library(rpart)

library(caret)
library(xgboost)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: lattice

Attaching package: ‘caret’

The following objects are masked from ‘package:Metrics’:

    precision, recall

The following object is masked from ‘package:purrr’:

    lift



ERROR: Error in library(xgboost): there is no package called ‘xgboost’


In [9]:
#cl_data <- read_csv("../datasets/Dataset 3 Beijing/Datacleaning/cleaned_beijing.csv")
cl_data <- read_csv("cleaned_beijing.csv")
cl_data <- cl_data[ -c(1) ]

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_double(),
  tradeTime = col_date(format = ""),
  buildingType = col_character(),
  renovationCondition = col_character(),
  buildingStructure = col_character(),
  elevator = col_character(),
  fiveYearsProperty = col_character(),
  subway = col_character(),
  district = col_character()
)
See spec(...) for full column specifications.


In [4]:
head(cl_data)

Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
116.4755,40.01952,1.111027e+12,2016-08-09,1464,106,415.0,31680,131.00,2,...,Tower,2005,Simplicit,Steel/Concrete,0.217,Has_Elevator,Ownership > 5y,Has_Subway,ChaoYang,56021
116.4539,39.88153,1.111027e+12,2016-07-28,903,126,575.0,43436,132.38,2,...,Tower,2004,Hardcover,Steel/Concrete,0.667,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,71539
116.5620,39.87714,1.111041e+12,2016-12-11,1271,48,1030.0,52021,198.00,3,...,Plate,2005,Simplicit,Steel/Concrete,0.500,Has_Elevator,Ownership > 5y,No_Subway,ChaoYang,48160
116.4380,40.07611,1.111043e+12,2016-09-30,965,138,297.5,22202,134.00,3,...,Tower,2008,Other,Steel/Concrete,0.273,Has_Elevator,Ownership > 5y,No_Subway,ChangPing,51238
116.4284,39.88623,1.111027e+12,2016-08-28,927,286,392.0,48396,81.00,2,...,Plate,1960,Rough,Mixed,0.333,No_elevator,Ownership < 5y,Has_Subway,DongCheng,62588
116.4663,39.99136,1.111027e+12,2016-07-22,861,57,275.6,52000,53.00,1,...,Plate,2005,Simplicit,Steel/Concrete,0.333,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,67738


In [5]:
summary(cl_data)

      Lng             Lat             Cid              tradeTime         
 Min.   :116.1   Min.   :39.63   Min.   :1.111e+12   Min.   :2002-06-01  
 1st Qu.:116.3   1st Qu.:39.89   1st Qu.:1.111e+12   1st Qu.:2013-12-26  
 Median :116.4   Median :39.93   Median :1.111e+12   Median :2015-09-27  
 Mean   :116.4   Mean   :39.95   Mean   :1.126e+12   Mean   :2015-04-30  
 3rd Qu.:116.5   3rd Qu.:40.00   3rd Qu.:1.111e+12   3rd Qu.:2016-08-12  
 Max.   :116.7   Max.   :40.25   Max.   :1.185e+14   Max.   :2018-01-28  
      DOM            followers         totalPrice         price       
 Min.   :   1.00   Min.   :   0.00   Min.   :  13.0   Min.   : 10000  
 1st Qu.:   7.00   1st Qu.:   0.00   1st Qu.: 206.0   1st Qu.: 28362  
 Median :   7.00   Median :   5.00   Median : 295.0   Median : 39132  
 Mean   :  18.54   Mean   :  17.25   Mean   : 350.6   Mean   : 44003  
 3rd Qu.:   8.00   3rd Qu.:  19.00   3rd Qu.: 428.0   3rd Qu.: 54348  
 Max.   :1677.00   Max.   :1143.00   Max.   :4900.0   Ma

In [5]:
unique(cl_data$constructionTime)

[1] 2005 2004 2008 1960 1997 2009 1991 2001 1990 2011 2000 1998 2010 1996 1993
[16] 2006 2002 2012 1989 2003 2007 1994 1984 1992 2014 1985 1999 1979 1981 1976
[31] 1982 1975 1983 1986 1995 1965 2013 1988 1987 2015 1955 1980 1978 1958 1970
[46] 1956 1977 1964 1963 1967 2016 1974 1973 1959 1954 1962 1966 1957 1972 1971
[61] 1953 1968 1961 1950 1952 1969

removing  Cid and total price. cid has too many factors (computer crashes when trained using as factor) 

In [10]:
data = cl_data[-c(3,7)]

# Data After Modifiation

In [7]:
head(data)

Lng,Lat,tradeTime,DOM,followers,price,square,livingRoom,drawingRoom,kitchen,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
116.4755,40.01952,2016-08-09,1464,106,31680,131.00,2,1,1,...,Tower,2005,Simplicit,Steel/Concrete,0.217,Has_Elevator,Ownership > 5y,Has_Subway,ChaoYang,56021
116.4539,39.88153,2016-07-28,903,126,43436,132.38,2,2,1,...,Tower,2004,Hardcover,Steel/Concrete,0.667,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,71539
116.5620,39.87714,2016-12-11,1271,48,52021,198.00,3,2,1,...,Plate,2005,Simplicit,Steel/Concrete,0.500,Has_Elevator,Ownership > 5y,No_Subway,ChaoYang,48160
116.4380,40.07611,2016-09-30,965,138,22202,134.00,3,1,1,...,Tower,2008,Other,Steel/Concrete,0.273,Has_Elevator,Ownership > 5y,No_Subway,ChangPing,51238
116.4284,39.88623,2016-08-28,927,286,48396,81.00,2,1,1,...,Plate,1960,Rough,Mixed,0.333,No_elevator,Ownership < 5y,Has_Subway,DongCheng,62588
116.4663,39.99136,2016-07-22,861,57,52000,53.00,1,0,1,...,Plate,2005,Simplicit,Steel/Concrete,0.333,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,67738


In [4]:
colnames(data)

[1] "Lng"                 "Lat"                 "tradeTime"          
 [4] "DOM"                 "followers"           "price"              
 [7] "square"              "livingRoom"          "drawingRoom"        
[10] "kitchen"             "bathRoom"            "floor"              
[13] "buildingType"        "constructionTime"    "renovationCondition"
[16] "buildingStructure"   "ladderRatio"         "elevator"           
[19] "fiveYearsProperty"   "subway"              "district"           
[22] "communityAverage"

In [9]:
str(data)

Classes 'tbl_df', 'tbl' and 'data.frame':	296350 obs. of  22 variables:
 $ Lng                : num  116 116 117 116 116 ...
 $ Lat                : num  40 39.9 39.9 40.1 39.9 ...
 $ tradeTime          : Date, format: "2016-08-09" "2016-07-28" ...
 $ DOM                : num  1464 903 1271 965 927 ...
 $ followers          : num  106 126 48 138 286 57 167 138 218 134 ...
 $ price              : num  31680 43436 52021 22202 48396 ...
 $ square             : num  131 132 198 134 81 ...
 $ livingRoom         : num  2 2 3 3 2 1 2 3 1 1 ...
 $ drawingRoom        : num  1 2 2 1 1 0 1 2 0 0 ...
 $ kitchen            : num  1 1 1 1 1 1 1 1 1 0 ...
 $ bathRoom           : num  1 2 3 1 1 1 1 2 1 0 ...
 $ floor              : num  26 22 4 21 6 8 6 22 10 23 ...
 $ buildingType       : chr  "Tower" "Tower" "Plate" "Tower" ...
 $ constructionTime   : num  2005 2004 2005 2008 1960 ...
 $ renovationCondition: chr  "Simplicit" "Hardcover" "Simplicit" "Other" ...
 $ buildingStructure  : chr  "Steel/Con

# Runing an analysis of variabce to see the significat factors

In [10]:
anova <- aov( price ~ . , data = data )

In [11]:
summary(anova)

                        Df    Sum Sq   Mean Sq  F value Pr(>F)    
Lng                      1 3.362e+12 3.362e+12  35190.1 <2e-16 ***
Lat                      1 2.696e+11 2.696e+11   2822.3 <2e-16 ***
tradeTime                1 3.956e+13 3.956e+13 414057.9 <2e-16 ***
DOM                      1 9.809e+11 9.809e+11  10266.3 <2e-16 ***
followers                1 1.049e+10 1.049e+10    109.8 <2e-16 ***
square                   1 3.485e+12 3.485e+12  36475.2 <2e-16 ***
livingRoom               1 3.904e+11 3.904e+11   4086.1 <2e-16 ***
drawingRoom              1 5.909e+10 5.909e+10    618.4 <2e-16 ***
kitchen                  1 1.609e+11 1.609e+11   1683.8 <2e-16 ***
bathRoom                 1 4.947e+11 4.947e+11   5178.2 <2e-16 ***
floor                    1 5.533e+11 5.533e+11   5791.1 <2e-16 ***
buildingType             3 5.894e+11 1.965e+11   2056.5 <2e-16 ***
constructionTime         1 5.432e+12 5.432e+12  56851.2 <2e-16 ***
renovationCondition      3 1.355e+12 4.517e+11   4727.3 <2e-16

# Spliting Data

In [11]:
train_index <- sample(1:nrow(data), 0.8 * nrow(data))
test_index <- setdiff(1:nrow(data), train_index)

# Build X_train, y_train, X_test, y_test
train <- data[train_index,]


test <- data[test_index,]


# Model creation

## model with linercombiantion of all facotors

In [13]:
model <- lm(price ~ . , data = train )

In [14]:
summary(model)


Call:
lm(formula = price ~ ., data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-90350  -6080   -615   5448 142676 

Coefficients:
                                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)                      2.745e+05  5.809e+04   4.726 2.29e-06 ***
Lng                             -3.306e+03  4.358e+02  -7.586 3.31e-14 ***
Lat                             -3.647e+03  4.994e+02  -7.302 2.84e-13 ***
tradeTime                        2.258e+01  5.150e-02 438.417  < 2e-16 ***
DOM                              3.586e+01  6.310e-01  56.835  < 2e-16 ***
followers                        8.860e+00  7.019e-01  12.623  < 2e-16 ***
square                          -6.179e+01  1.240e+00 -49.840  < 2e-16 ***
livingRoom                       1.404e+02  4.273e+01   3.287  0.00101 ** 
drawingRoom                      1.208e+03  5.136e+01  23.519  < 2e-16 ***
kitchen                         -8.640e+02  2.112e+02  -4.090 4.31e-05 ***
bathRoom                       

mean funtion not working had to imporvised now 

In [15]:
p = predict(model, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 9761.893

## model with limited factors for initial testing

In [16]:
model_limited <- lm(price ~ square + buildingType +constructionTime + subway + district , data = train )


In [17]:
summary(model_limited)


Call:
lm(formula = price ~ square + buildingType + constructionTime + 
    subway + district, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-71747 -11248  -3453   9138 114062 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)              6.921e+04  1.052e+04   6.578 4.77e-11 ***
square                  -3.099e+01  1.106e+00 -28.026  < 2e-16 ***
buildingTypePlate       -3.305e+04  3.160e+03 -10.458  < 2e-16 ***
buildingTypePlate/Tower -3.235e+04  3.161e+03 -10.234  < 2e-16 ***
buildingTypeTower       -3.520e+04  3.161e+03 -11.137  < 2e-16 ***
constructionTime        -3.823e-01  5.052e+00  -0.076   0.9397    
subwayNo_Subway         -3.807e+03  7.933e+01 -47.991  < 2e-16 ***
districtChaoYang         1.292e+04  1.292e+02  99.939  < 2e-16 ***
districtDaXing           1.874e+03  4.328e+02   4.330 1.49e-05 ***
districtDongCheng        2.930e+04  1.978e+02 148.099  < 2e-16 ***
districtFangShang       -1.581e+03  4.021e+02  -3.933 8

In [18]:
p = predict(model_limited, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 17637.94

'price'
'square'
'livingRoom'
'drawingRoom'
'kitchen'
'bathRoom'
'floor'
'buildingType'
'constructionTime'
'renovationCondition'
'buildingStructure'
'ladderRatio'
'elevator'
'fiveYearsProperty'
'subway'
'district'## tree model

In [19]:
fit <- rpart(price ~ square + livingRoom + drawingRoom + kitchen + bathRoom + floor + buildingType + 
             constructionTime + renovationCondition + buildingStructure + elevator+ subway+ district, data = train ,method="anova")

In [20]:
summary(fit)

Call:
rpart(formula = price ~ square + livingRoom + drawingRoom + kitchen + 
    bathRoom + floor + buildingType + constructionTime + renovationCondition + 
    buildingStructure + elevator + subway + district, data = train, 
    method = "anova")
  n= 237080 

          CP nsplit rel error    xerror        xstd
1 0.23749922      0 1.0000000 1.0000080 0.004163942
2 0.06345670      1 0.7625008 0.7625198 0.002961263
3 0.05575580      2 0.6990441 0.6990673 0.002821824
4 0.03671490      3 0.6432883 0.6433195 0.002672968
5 0.01367052      4 0.6065734 0.6066069 0.002615369
6 0.01133829      5 0.5929029 0.5929536 0.002522452
7 0.01000000      6 0.5815646 0.5816146 0.002520933

Variable importance
           district renovationCondition   buildingStructure              subway 
                 68                  22                   2                   2 
              floor            elevator        buildingType    constructionTime 
                  2                   1                   

In [21]:
p = predict(fit, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 16613.65

In [23]:
save(fit, file = "models/limited_model.rda")
save(fit, file = "../house-price-prediction-app/shinyApp/models/limited_model.rda")

## Random forest

crashes computer uses all ram

## Feature analysis

In [60]:
summary(train$square); class(train$square); length(unique(train$square))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   7.37   57.90   73.80   82.49   98.03  640.00 

[1] "numeric"

[1] 16747

In [41]:
unique(train$livingRoom); class(train$livingRoom)

[1] 2 1 3 4 5 6 7 0 8

[1] "numeric"

In [42]:
unique(train$drawingRoom); class(train$drawingRoom)

[1] 2 0 1 3 4 5

[1] "numeric"

In [43]:
unique(train$kitchen); class(train$kitchen)

[1] 1 0 2 3

[1] "numeric"

In [44]:
unique(train$bathRoom); class(train$bathRoom)

[1] 1 2 3 4 0 6 5 7

[1] "numeric"

In [46]:
summary(train$floor); class(train$floor)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    6.00   11.00   13.17   19.00   63.00 

[1] "numeric"

In [47]:
unique(train$buildingType); class(train$buildingType)

[1] "Tower"       "Plate/Tower" "Plate"       "Bungalow"

[1] "character"

In [48]:
unique(train$buildingStructure); class(train$buildingStructure)

[1] "Steel/Concrete" "Mixed"          "Brick/Concrete" "Brick/Wood"    
[5] "Steel"          "Unavailable"

[1] "character"

In [49]:
summary(train$constructionTime); class(train$constructionTime)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1950    1994    2001    1999    2006    2016 

[1] "numeric"

In [50]:
unique(train$renovationCondition); class(train$renovationCondition)

[1] "Hardcover" "Other"     "Simplicit" "Rough"

[1] "character"

In [51]:
unique(train$elevator); class(train$elevator)

[1] "Has_Elevator" "No_elevator"

[1] "character"

In [52]:
unique(train$subway); class(train$subway)

[1] "Has_Subway" "No_Subway"

[1] "character"

In [53]:
unique(train$district); class(train$district)

[1] "ChaoYang"    "HaiDian"     "FengTai"     "DongCheng"   "XiCheng"    
 [6] "FaXing"      "ChangPing"   "ShiJingShan" "MenTouGou"   "ShunYi"     
[11] "DaXing"      "TongZhou"    "FangShang"

[1] "character"

## Extended liner Model

In [54]:
str(data)

Classes 'tbl_df', 'tbl' and 'data.frame':	296350 obs. of  22 variables:
 $ Lng                : num  116 116 117 116 116 ...
 $ Lat                : num  40 39.9 39.9 40.1 39.9 ...
 $ tradeTime          : Date, format: "2016-08-09" "2016-07-28" ...
 $ DOM                : num  1464 903 1271 965 927 ...
 $ followers          : num  106 126 48 138 286 57 167 138 218 134 ...
 $ price              : num  31680 43436 52021 22202 48396 ...
 $ square             : num  131 132 198 134 81 ...
 $ livingRoom         : num  2 2 3 3 2 1 2 3 1 1 ...
 $ drawingRoom        : num  1 2 2 1 1 0 1 2 0 0 ...
 $ kitchen            : num  1 1 1 1 1 1 1 1 1 0 ...
 $ bathRoom           : num  1 2 3 1 1 1 1 2 1 0 ...
 $ floor              : num  26 22 4 21 6 8 6 22 10 23 ...
 $ buildingType       : chr  "Tower" "Tower" "Plate" "Tower" ...
 $ constructionTime   : num  2005 2004 2005 2008 1960 ...
 $ renovationCondition: chr  "Simplicit" "Hardcover" "Simplicit" "Other" ...
 $ buildingStructure  : chr  "Steel/Con

In [24]:
model_ext<- lm(price ~ square + livingRoom + drawingRoom + kitchen + bathRoom + floor + buildingType + 
             constructionTime + renovationCondition + buildingStructure + elevator+ subway+ district , data = train )

In [25]:
summary(model_ext)


Call:
lm(formula = price ~ square + livingRoom + drawingRoom + kitchen + 
    bathRoom + floor + buildingType + constructionTime + renovationCondition + 
    buildingStructure + elevator + subway + district, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-78859 -10688  -2503   8004 121481 

Coefficients:
                                  Estimate Std. Error  t value Pr(>|t|)    
(Intercept)                     271872.156  11868.828   22.906  < 2e-16 ***
square                             -86.526      1.997  -43.332  < 2e-16 ***
livingRoom                        1197.110     70.314   17.025  < 2e-16 ***
drawingRoom                       -263.154     84.174   -3.126  0.00177 ** 
kitchen                           4631.205    347.980   13.309  < 2e-16 ***
bathRoom                          3181.315    117.538   27.066  < 2e-16 ***
floor                              139.477      8.387   16.629  < 2e-16 ***
buildingTypePlate               -27097.134   3712.295   -7.299 2.90e-1

In [26]:
p = predict(model_ext, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 16247.09

In [27]:
save(model_ext, file = "models/extended_model.rda")
save(model_ext, file = "../house-price-prediction-app/shinyApp/models/extended_model.rda")

## converting some (num) columns to factors

In [12]:
str(data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	296350 obs. of  22 variables:
 $ Lng                : num  116 116 117 116 116 ...
 $ Lat                : num  40 39.9 39.9 40.1 39.9 ...
 $ tradeTime          : Date, format: "2016-08-09" "2016-07-28" ...
 $ DOM                : num  1464 903 1271 965 927 ...
 $ followers          : num  106 126 48 138 286 57 167 138 218 134 ...
 $ price              : num  31680 43436 52021 22202 48396 ...
 $ square             : num  131 132 198 134 81 ...
 $ livingRoom         : num  2 2 3 3 2 1 2 3 1 1 ...
 $ drawingRoom        : num  1 2 2 1 1 0 1 2 0 0 ...
 $ kitchen            : num  1 1 1 1 1 1 1 1 1 0 ...
 $ bathRoom           : num  1 2 3 1 1 1 1 2 1 0 ...
 $ floor              : num  26 22 4 21 6 8 6 22 10 23 ...
 $ buildingType       : chr  "Tower" "Tower" "Plate" "Tower" ...
 $ constructionTime   : num  2005 2004 2005 2008 1960 ...
 $ renovationCondition: chr  "Simplicit" "Hardcover" "Simplicit" "Other" ...
 $ buildingStructure  : chr  "Steel/Con

In [31]:
data$livingRoom <- as.factor(data$livingRoom)
data$drawingRoom <- as.factor(data$drawingRoom)
data$kitchen <- as.factor(data$kitchen)
data$bathRoom <- as.factor(data$bathRoom)
data$floor <- as.factor(data$floor) # this might cause problems 

In [32]:
str(data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	296350 obs. of  22 variables:
 $ Lng                : num  116 116 117 116 116 ...
 $ Lat                : num  40 39.9 39.9 40.1 39.9 ...
 $ tradeTime          : Date, format: "2016-08-09" "2016-07-28" ...
 $ DOM                : num  1464 903 1271 965 927 ...
 $ followers          : num  106 126 48 138 286 57 167 138 218 134 ...
 $ price              : num  31680 43436 52021 22202 48396 ...
 $ square             : num  131 132 198 134 81 ...
 $ livingRoom         : Factor w/ 9 levels "0","1","2","3",..: 3 3 4 4 3 2 3 4 2 2 ...
 $ drawingRoom        : Factor w/ 6 levels "0","1","2","3",..: 2 3 3 2 2 1 2 3 1 1 ...
 $ kitchen            : Factor w/ 4 levels "0","1","2","3": 2 2 2 2 2 2 2 2 2 1 ...
 $ bathRoom           : Factor w/ 8 levels "0","1","2","3",..: 2 3 4 2 2 2 2 3 2 1 ...
 $ floor              : Factor w/ 40 levels "1","2","3","4",..: 26 22 4 21 6 8 6 22 10 23 ...
 $ buildingType       : chr  "Tower" "Tower" "Plate" "Tower" ...
 $ con

In [33]:
train <- data[train_index,]
test <- data[test_index,]

In [34]:
model <- lm(price ~ livingRoom + drawingRoom + kitchen + bathRoom + floor + buildingType + 
             constructionTime + renovationCondition + buildingStructure + elevator+ subway+ district , data = train )

In [35]:
summary(model)


Call:
lm(formula = price ~ livingRoom + drawingRoom + kitchen + bathRoom + 
    floor + buildingType + constructionTime + renovationCondition + 
    buildingStructure + elevator + subway + district, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-62949 -10688  -2509   7965 124198 

Coefficients:
                                  Estimate Std. Error  t value Pr(>|t|)    
(Intercept)                     367816.802  12701.944   28.958  < 2e-16 ***
livingRoom1                       2164.405   4047.781    0.535  0.59285    
livingRoom2                       1159.886   4047.557    0.287  0.77445    
livingRoom3                       1043.591   4048.396    0.258  0.79658    
livingRoom4                        187.040   4056.180    0.046  0.96322    
livingRoom5                      -4341.791   4101.276   -1.059  0.28976    
livingRoom6                      -5357.366   4278.203   -1.252  0.21048    
livingRoom7                      -5819.916   5836.323   -0.997  0.31867    
liv

In [36]:
p = predict(model, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 16120.74

minor perfoamce imporvemnt, even after removing square